<a href="https://colab.research.google.com/github/bagrezildo/poke-scraper/blob/main/serebii_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import bs4
import requests
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
def abrir_imagem(url):
    # Faz o download da imagem a partir do link
    response = requests.get(url)

    # Verifica se o download foi bem-sucedido
    if response.status_code == 200:
        # Abre a imagem utilizando a biblioteca Pillow
        imagem = Image.open(BytesIO(response.content))
        return imagem
    else:
        print("Falha ao baixar a imagem.")
        return None

#Definição de funções de busca

In [3]:
def find_other_names(soup): ##Extrai as variacoes idiomaticas do nome dos pokemons
  tag = soup.find_all('td', class_='fooinfo')

  names=tag[2].text.replace(" ","").split('\n')

  other_names = [[],[]]

  for j in names[1:-1]:
      other_names[0].append(j.split(":")[0])
      other_names[1].append(j.split(":")[1])

  other_names = dict(zip(other_names[0], other_names[1]))

  return other_names

def find_no(soup): ##Extrai o numero dos pokemons em cada uma das pokedex
  tag = soup.find_all('td', class_='fooinfo')

  num=tag[3].text.replace(" ","").split("\n")

  no = [[],[]]

  for i in num[:-1]:
    no[0].append(i.split(":")[0])
    no[1].append(i.split(":")[1])

  no = dict(zip(no[0], no[1]))
  return no

def find_gender_ratio(soup): ##Extrai a taxa de encontro de cada genero
  tag = soup.find_all('td', class_='fooinfo')
  gender = tag[4].text
  male_index=tag[4].text.find("Male")
  female_index=tag[4].text.find("Female")

  gender_ratio={
      'male': gender[male_index:female_index],
      'female': gender[female_index:]
  }
  return gender_ratio

In [4]:
def find_sprites(soup, id):
    tag = soup.find_all('td', class_='fooinfo')

    icon = f'https://serebii.net/pokedex-xy/icon/{id}.png'
    default = f'https://www.serebii.net/xy/pokemon/{id}.png'
    shiny = f'https://www.serebii.net/Shiny/XY/{id}.png'
    sprites = {
        'icon': icon,
        'default': default,
        'shiny': shiny
    }
    return sprites

In [5]:
def find_height(soup):
  tag = soup.find_all('td', class_='fooinfo')

  h = tag[6].text        ##recupera os valores de altura
  h = h.replace("\r","") ##remove caracteres
  h = h.replace("\t","") ##remove caracteres
  h = h.split("\n")      ##separa a string

  ####armazena em um dicionario ft: feet, m: metros
  height = {
      'ft': h[0],
      'm': h[1]
  }
  return height

def find_weight(soup):
  tag = soup.find_all('td', class_='fooinfo')

  w = tag[7].text         ##recupera os valores de altura
  w = w.replace("\r","")  ##remove caracteres
  w = w.replace("\t","")  ##remove caracteres
  w = w.split("\n")       ##separa a string

  ####armazena em um dicionario lbs: libras, m: kilos
  weight = {
      'lbs':w[0],
      'kg':w[1]
  }

  return weight

In [6]:
def find_egg_group(soup):
    tag = soup.find_all('td', class_='fooinfo')
    egg = tag[15].find_all('tr')
    groups = []
    for i in egg:
        groups.append(i.find('option').text)
    list_keys = []
    for i in range(len(groups)):
      list_keys.append(f"egg_group_{i+1}")
    egg_group = dict(zip(list_keys, groups))
    return egg_group

In [7]:
##LOCATIONS
def find_locations(soup):
  tag = soup.find_all('td', class_='fooinfo')
  locations = {
      'x': tag[17].text,
      'y': tag[18].text,
      'or': tag[19].text,
      'as': tag[20].text
  }
  return locations

###FLAVOR TEXT
def find_flavor_text(soup):
  tag = soup.find_all('td', class_='fooinfo')

  flavor_text = {
      'x': tag[22].text,
      'y': tag[23].text,
      'or': tag[24].text,
      'as': tag[25].text
  }
  return flavor_text

In [8]:
def find_damage_taken(soup):
    table = soup.find_all(class_='dextable')
    types = ['normal', 'fire', 'water', 'eletric', 'grass',
            'ice', 'fighting', 'poison', 'ground', 'flying',
            'psychic', 'bug', 'rock', 'ghost', 'dragon', 'dark', 'steel', 'fairy']
    #DAMAGE TAKEN PER TYPE
    dmg = table[3].find_all('td', string=True, class_=('footype'))
    damage = []
    for i in dmg:
        damage.append(i.text)
    #print(damage)

    damage_taken = dict(zip(types,damage))
    return damage_taken

In [9]:
## find moves. TEM QUE REFAZER, POKEMONS SEM HOENN LVL UP CRASHAM A FUNCAO
def find_moves(soup):
    table = soup.find_all(class_='dextable')

    tr1 = table[8].find_all('tr')
    descript=[]
    moves_xy = {}
    j=0

    arg = 'colspan="8"'
    for i in tr1[2:]:
        if arg in str(i):
            descript.append(i.text.strip())
            #print(i.text)

        else:
            #print(i)
            attack = {
            'lvl': i.find_all('td')[0].text,
            'attack_name': i.find_all('td')[1].text,
            'attack_type': str(i.find_all('img')[0]).split("/",)[3].split(".")[0],
            'category': str(i.find_all('img')[1]).split("/",)[3].split(".")[0],
            'att': i.find_all('td')[4].text,
            'acc': i.find_all('td')[5].text,
            'pp': i.find_all('td')[6].text,
            'effect': i.find_all('td')[7].text,
            }
            moves_xy.update({j:attack})
            j+=1


    for chave, valor in zip(moves_xy.keys(), descript):
        moves_xy[chave]['description'] = valor

    tr2 = table[9].find_all('tr')
    descript=[]
    moves_oras = {}
    j=0

    for i in tr2[2:]:
        if 'colspan="8"' in str(i):
            descript.append(i.text)
            #print(i.text)

        else:
            #print(i)
            attack = {
                'lvl': i.find_all('td')[0].text,
                'attack_name': i.find_all('td')[1].text,
                'attack_type': str(i.find_all('img')[0]).split("/",)[3].split(".")[0],
                'category': str(i.find_all('img')[1]).split("/",)[3].split(".")[0],
                'att': i.find_all('td')[4].text,
                'acc': i.find_all('td')[5].text,
                'pp': i.find_all('td')[6].text,
                'effect': i.find_all('td')[7].text,
            }
            moves_oras.update({j:attack})
            j+=1


    for chave, valor in zip(moves_oras.keys(), descript):
        moves_oras[chave]['description'] = valor
    #print(moves_oras)

    tr3 = table[10].find_all('tr')

    descript=[]
    moves_tmhm = {}
    j=0



    for i in tr3[2:]:

        if 'colspan="8"' in str(i):
            descript.append(i.text)
            #print(i.text)

        else:
            #print(i)
            attack = {
            'tm/hm#': i.find_all('td')[0].text,
            'attack_name': i.find_all('td')[1].text,
            'attack_type': str(i.find_all('img')[0]).split("/",)[3].split(".")[0],
            'category': str(i.find_all('img')[1]).split("/",)[3].split(".")[0],
            'att': i.find_all('td')[4].text,
            'acc': i.find_all('td')[5].text,
            'pp': i.find_all('td')[6].text,
            'effect': i.find_all('td')[7].text,
            }
            moves_tmhm.update({j:attack})
            j+=1


    for chave, valor in zip(moves_tmhm.keys(), descript):
        moves_tmhm[chave]['description'] = valor
    #print(moves_tmhm)
    moves = {
        'xy': moves_xy,
        'oras': moves_oras,
        'tmhm': moves_tmhm
    }

    return moves

In [10]:
##DICIONARIO STATS
def find_stats(soup):
    for i in soup.find_all('td', class_="fooevo"):
      if 'colspan="8"><b>Stats' in str(i) and 'Mega Evolution' not in str(i):
        table = i.parent.parent
    tab = table.find_all('tr')
    list_keys = ['hp', 'attack', 'defense', 'sp_attack',
                 'sp_defense', 'speed']

    base_stats = []
    base_stats_total = 0
    for i in tab[2]:
        if "Total" in str(i):
            #print(str(i).strip('<td class="fooinfo" colspan="2" width="14%"> Base Stats - Total: </td>'))
            base_stats_total = str(i).strip('<td class="fooinfo" colspan="2" width="14%"> Base Stats - Total: </td>')
        elif '\n' not in i.text:
            base_stats.append(i.text)
    #print(base_stats_total)
    base_stats = dict(zip(list_keys, base_stats))
    #base_stats

    hin_lv50 = []
    for i in tab[3].find_all(class_="foohin"):
        if "center" in str(i):
            hin_lv50.append(i.text)

    hin_lv100 = []
    for i in tab[4].find_all(class_="foohin"):
        if "center" in str(i):
            hin_lv100.append(i.text)

    hindering_nature = {
        'lv50':dict(zip(list_keys,hin_lv50)),
        'lv100':dict(zip(list_keys,hin_lv100)),
    }
    #hindering_nature


    neu_lv50 = []
    for i in tab[5].find_all(class_="fooinfo"):
        if "center" in str(i):
            neu_lv50.append(i.text)

    neu_lv100 = []
    for i in tab[6].find_all(class_="fooinfo"):
        if "center" in str(i):
            neu_lv100.append(i.text)

    neutral_nature = {
        'lv50':dict(zip(list_keys,neu_lv50)),
        'lv100':dict(zip(list_keys,neu_lv100)),
    }
    #neutral_nature


    ben_lv50 = []
    for i in tab[7].find_all(class_="fooben"):
        if "center" in str(i):
            ben_lv50.append(i.text)

    ben_lv100 = []
    for i in tab[8].find_all(class_="fooben"):
        if "center" in str(i):
            ben_lv100.append(i.text)

    beneficial_nature = {
        'lv50':dict(zip(list_keys,ben_lv50)),
        'lv100':dict(zip(list_keys,ben_lv100)),
    }
    #beneficial_nature

    stats={
        'base_stats_total': base_stats_total,
        'base_stats': base_stats,
        'max_stats':{
            'hindering_nature': hindering_nature,
            'neutral_nature': neutral_nature,
            'beneficial_nature': beneficial_nature
        }
    }
    return stats

In [11]:
def find_pokemon(soup, id):

    tag = soup.find_all('td', class_='fooinfo')

    name = tag[1].text
    sprites = find_sprites(soup, id)
    other_names = find_other_names(soup)
    no = find_no(soup)
    gender_ratio = find_gender_ratio(soup)
    classification = tag[5].text
    height = find_height(soup)
    weight = find_weight(soup)
    capture_rate = tag[8].text
    egg_steps = tag[9].text
    abilities = tag[10].text.split('\n')
    experience_growth = tag[11].text
    base_hapiness = tag[12].text
    ev_earned = tag[13].text
    sky_battle = tag[14].text.strip('\n\t\r')
    egg_group = find_egg_group(soup)
    locations = find_locations(soup)
    flavor_text = find_flavor_text(soup)
    damage_taken = find_damage_taken(soup)
    #moves = find_moves(soup)
    stats = find_stats(soup)


    pokemon = {
        'id': id,
        'name': name,
        'sprite': sprites,
        'other_names': other_names,
        'no': no,
        'gender_ratio': gender_ratio,
        'classification': classification,
        'height': height,
        'weight': weight,
        'capture_rate': capture_rate,
        'egg_steps': egg_steps,
        'abilities': abilities,
        'experience_growth': experience_growth,
        'base_hapiness': base_hapiness,
        'ev_earned': ev_earned,
        'sky_battle': sky_battle,
        'egg_group': egg_group,
        'locations': locations,
        'flavor_text': flavor_text,
        'damage_taken': damage_taken,
        #'moves': moves,
        'stats': stats
    }
    return pokemon

#Definição de funções de imprimir

In [12]:
def print_classification(pokemon):
  classification = f"{'Classification':^15} | {pokemon['classification']:^{len(pokemon['classification'])}}"
  print(f"{'_'*len(classification)} ")
  print(classification)
  print(f"{'_'*len(classification)} ")

In [13]:
def imprime_pokemon(pokemon):
  display(abrir_imagem(pokemon['sprite']['icon']))

  mon = f"#{pokemon['id']:^4} | {pokemon['name']:^{len(pokemon['name'])}}"
  print(f"{'_'*len(mon)} ")
  print(mon)
  print(f"{'_'*len(mon)} ")


  display(abrir_imagem(pokemon['sprite']['default']))

  print_no_dex(pokemon)

  print_classification(pokemon)

  print_abilities(pokemon)

  print_egg_groups(pokemon)
  print_locations(pokemon)
  print_flavor_text(pokemon)
  print_damage_taken(pokemon)
  print_stats(pokemon)
  print_moves(pokemon)

In [14]:
def print_moves(pokemon):
  moves = pokemon['moves']
  #att = list(moves['xy'][0].values())
  #Level	Attack Name	Type	Cat.	Att.	Acc.	PP	Effect %

  for dex in moves:
    print(f"{'_'*103} ")
    print(f"{dex:^90}")
    print(f"{'_'*103} ")
    for move in list(moves[dex].values()):
      ataque = list(move.values())
      print(f"{ataque[0]:^5} | {ataque[1]:^20} | {ataque[2]:^9} | {ataque[3]:^9} | {ataque[4]:^10} | {ataque[5]:^11} | {ataque[6]:^9} | {ataque[7]:^9}")
      print(f"{'_'*5} | {'_'*20} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} | {'_'*9} ")
      print(f"{' ':^28} | {ataque[8]:^60}")
      print(f"{'_'*28} | {'_'*72} ")


In [15]:
def print_no_dex(pokemon):
  types = list(pokemon['no'].keys())
  values = list(pokemon['no'].values())
  print(f"{'_'*35} ")
  print(f"{'Dex':^35}")
  print(f"{'_'*35} ")

  for i in range(len(types)):
    print(f"{types[i]:^14} | {values[i]:^18}")
    print(f"{'_'*14} | {'_'*18} ")


In [16]:
def print_egg_groups(pokemon):
  values = list(pokemon['egg_group'].values())

  texto = f"{'Egg Groups':^12} |"
  texto2 = ' '.join([f"{i:^12} " for i in values])
  texto = texto + texto2

  print(f"{'_'*len(texto)} ")
  print(texto)
  print(f"{'_'*len(texto)} ")


In [17]:
def print_locations(pokemon):
  types = ['X', 'Y', 'Omega Ruby', 'Alpha Saphire']
  values = list(pokemon['locations'].values())

  print(f"{'_'*35} ")
  print(f"{'Locations':^35}")
  print(f"{'_'*35} ")
  print(f"{types[0]:^14} | {values[0]:^18}")
  print(f"{'_'*14} | {'_'*18} ")
  print(f"{types[1]:^14} | {values[1]:^18} ")
  print(f"{'_'*14} | {'_'*18} ")
  print(f"{types[2]:^14} | {values[2]:^18}")
  print(f"{'_'*14} | {'_'*18} ")
  print(f"{types[3]:^14} | {values[3]:^18}")
  print(f"{'_'*14} | {'_'*18} ")


In [18]:
def print_abilities(pokemon):
  types = ['X', 'Y', 'Omega Ruby', 'Alpha Saphire']
  values = pokemon['abilities'][1:]

  print(f"{'_'*200} ")
  print(f"{'Abilities':^14}")
  print(f"{'_'*200} ")
  for i in range(len(values)):
    print(f"{values[i]:<200}")
    print(f"{'_'*200} ")



In [19]:
def print_flavor_text(pokemon):
  types = ['X', 'Y', 'Omega Ruby', 'Alpha Saphire']
  values = list(pokemon['flavor_text'].values())

  print(f"{'_'*200} ")
  print(f"{'Flavor Text':^14}")
  print(f"{'_'*200} ")
  for i in range(len(types)):
    print(f"{types[i]:^14} | {values[i]:<180}")
    print(f"{'_'*14} | {'_'*183} ")



In [20]:
def print_damage_taken(pokemon):
  types = list(pokemon['damage_taken'].keys())
  values = list(pokemon['damage_taken'].values())

  print(f"{'_'*72} ")
  print(f"{'Damage Taken':^72}")
  print(f"{'_'*72} ")
  print(f"{types[0]:^9} | {values[0]:^9} | {types[6]:^9} | {values[6]:^10} | {types[12]:^11} | {values[12]:^9}")
  print(f"{'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{types[1]:^9} | {values[1]:^9} | {types[7]:^9} | {values[7]:^10} | {types[13]:^11} | {values[13]:^9}")
  print(f"{'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{types[2]:^9} | {values[2]:^9} | {types[8]:^9} | {values[8]:^10} | {types[14]:^11} | {values[14]:^9}")
  print(f"{'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{types[3]:^9} | {values[3]:^9} | {types[9]:^9} | {values[9]:^10} | {types[15]:^11} | {values[15]:^9}")
  print(f"{'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{types[4]:^9} | {values[4]:^9} | {types[10]:^9} | {values[10]:^10} | {types[16]:^11} | {values[16]:^9}")
  print(f"{'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{types[5]:^9} | {values[5]:^9} | {types[11]:^9} | {values[11]:^10} | {types[17]:^11} | {values[17]:^9}")
  print(f"{'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")


In [21]:
def print_stats(pokemon):

  hin_lv50 = list(pokemon['stats']['max_stats']['hindering_nature']['lv50'].values())

  hin_lv100 = list(pokemon['stats']['max_stats']['hindering_nature']['lv100'].values())

  ###NEUTRAL
  neu_lv50 = list(pokemon['stats']['max_stats']['neutral_nature']['lv50'].values())

  neu_lv100 = list(pokemon['stats']['max_stats']['neutral_nature']['lv100'].values())

  ###BENEFICIAL
  ben_lv50 = list(pokemon['stats']['max_stats']['beneficial_nature']['lv50'].values())


  ben_lv100 = list(pokemon['stats']['max_stats']['beneficial_nature']['lv100'].values())

  total ='Base Stats - Total: '+pokemon['stats']['base_stats_total']

  base = list(pokemon['stats']['base_stats'].values())


  print(f"{'_'*100} ")
  print(f"{'Stats':^90}")
  print(f"{'_'*100} ")
  print(f"{' ':^25} | {'HP':^9} | {'Attack':^9} | {'Defense':^9} | {'Sp. Attack':^10} | {'Sp. Defense':^11} | {'Speed':^9}")
  print(f"{'_'*25} | {'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{total:^25} | {base[0]:^9} | {base[1]:^9} | {base[2]:^9} | {base[3]:^10} | {base[4]:^11} | {base[5]:^9}")
  print(f"{'_'*25} | {'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{'Max Stats':^16} | {'Lv.50':^6} | {hin_lv50[0]:^9} | {hin_lv50[1]:^9} | {hin_lv50[2]:^9} | {hin_lv50[3]:^10} | {hin_lv50[4]:^11} | {hin_lv50[5]:^9}")
  print(f"{' '*16} | {'_'*6} | {'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{'Hindering Nature':^16} | {'Lv.100':^6} | {hin_lv100[0]:^9} | {hin_lv100[1]:^9} | {hin_lv100[2]:^9} | {hin_lv100[3]:^10} | {hin_lv100[4]:^11} | {hin_lv100[5]:^9}")
  print(f"{'_'*25} | {'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{'Max Stats':^16} | {'Lv.50':^6} | {neu_lv50[0]:^9} | {neu_lv50[1]:^9} | {neu_lv50[2]:^9} | {neu_lv50[3]:^10} | {neu_lv50[4]:^11} | {neu_lv50[5]:^9}")
  print(f"{' '*16} | {'_'*6} | {'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{'Neutral Nature':^16} | {'Lv.100':^6} | {neu_lv100[0]:^9} | {neu_lv100[1]:^9} | {neu_lv100[2]:^9} | {neu_lv100[3]:^10} | {neu_lv100[4]:^11} | {neu_lv100[5]:^9}")
  print(f"{'_'*25} | {'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{'Max Stats':^16} | {'Lv.50':^6} | {ben_lv50[0]:^9} | {ben_lv50[1]:^9} | {ben_lv50[2]:^9} | {ben_lv50[3]:^10} | {ben_lv50[4]:^11} | {ben_lv50[5]:^9}")
  print(f"{' '*16} | {'_'*6} | {'_'*9} | {'_'*9} | {'_'*9} | {'_'*10} | {'_'*11} | {'_'*9} ")
  print(f"{'Benefical Nature':^16} | {'Lv.100':^6} | {ben_lv100[0]:^9} | {ben_lv100[1]:^9} | {ben_lv100[2]:^9} | {ben_lv100[3]:^10} | {ben_lv100[4]:^11} | {ben_lv100[5]:^9}")




#Executando

In [33]:
first = 100
last = 105
URLS = ['https://serebii.net/pokedex-xy/{}.shtml'.format(str(x).zfill(3))
                for x in range(first, last+1)]

pokedex_xy = {}
for url in URLS:

    id = url.strip('https://serebii.net/pokedex-xy/')
    id = id.strip('.shtml')

    print(f'To pegando o {id} em {url}')
    data = requests.get(url)

    soup = bs4.BeautifulSoup(data.text, 'html.parser')
    tag = soup.find_all('td', class_='fooinfo')

    moves(soup)

    pokemon = find_pokemon(soup, id)

    pokedex_xy.update({id:pokemon})
##o erro das moves é que nem todos os pokemons tem uma tabela de lvl de hoenn, tem que fazer casos em que nao tem.

To pegando o 100 em https://serebii.net/pokedex-xy/100.shtml
{'lvl': '—', 'attack_name': 'Charge', 'attack_type': 'electric', 'category': 'other', 'att': '--', 'acc': '--', 'pp': '20', 'effect': '--'}
{'lvl': '5', 'attack_name': 'Tackle', 'attack_type': 'normal', 'category': 'physical', 'att': '50', 'acc': '100', 'pp': '35', 'effect': '--'}
{'lvl': '8', 'attack_name': 'Sonic Boom', 'attack_type': 'normal', 'category': 'special', 'att': '??', 'acc': '90', 'pp': '20', 'effect': '--'}
{'lvl': '10', 'attack_name': 'Eerie Impulse', 'attack_type': 'electric', 'category': 'other', 'att': '--', 'acc': '100', 'pp': '15', 'effect': '--'}
{'lvl': '12', 'attack_name': 'Spark', 'attack_type': 'electric', 'category': 'physical', 'att': '65', 'acc': '100', 'pp': '20', 'effect': '30'}
{'lvl': '15', 'attack_name': 'Rollout', 'attack_type': 'rock', 'category': 'physical', 'att': '30', 'acc': '90', 'pp': '20', 'effect': '--'}
{'lvl': '19', 'attack_name': 'Screech', 'attack_type': 'normal', 'category': 'o

In [28]:
id = '258'
url = f'https://serebii.net/pokedex-xy/{id}.shtml'
data = requests.get(url)
soup = bs4.BeautifulSoup(data.text, 'html.parser')
table = soup.find_all(class_='dextable')
#tab = table[15].find_all('tr')
pokemon = find_pokemon(soup, id)

#icon
#no name
#sprite
#dex numbers
#gender ratio
#height e weight
#abilities
#egg_group
#locations
#flavor text as
#damage taken
#stats {tota e base stats}

In [ ]:
for pokemon in pokedex_xy:
  imprime_pokemon(pokedex_xy[pokemon])

In [32]:
def moves(soup):
  table = soup.findAll(class_='dextable')
  arg = 'colspan="8"'
  moves = []
  descript = []
  index = 0
  for i in table:
    #print(i.find('tr'))
    if ('Generation' in str(i.find('tr'))) or ('X &amp; Y Level Up' in str(i.find('tr'))):
      #print(i)
      for j in i.findAll('tr')[2:]:
        if arg in str(j):
          descript.append(j.text)

        else:
          #print('comeca', j.findAll('td')[4].text, 'termina')
          attack = {
                'lvl': j.findAll('td')[0].text,
                'attack_name': j.findAll('td')[1].text,
                'attack_type': str(j.find_all('img')[0]).split("/",)[3].split(".")[0],
                'category': str(j.find_all('img')[1]).split("/",)[3].split(".")[0],
                'att': j.find_all('td')[4].text,
                'acc': j.find_all('td')[5].text,
                'pp': j.find_all('td')[6].text,
                'effect': j.find_all('td')[7].text,
                }
          moves.append(attack)
          index+=1
          print(attack)

#for chave, valor in zip(moves.keys(), descript):
 #       moves[chave]['description'] = valor

In [38]:
table = soup.findAll(class_='dextable')
arg = 'colspan="8"'
moves_xy = {}
moves_oras = {}
moves_tm = {}
descript = []
index = 0
string = ['Generation VI Level Up', 'X &amp; Y Level Up']
for i in table:
  #print(i.find('tr'))

  for s in string if s in str(i.find('tr')):
    #print(i)
    for j in i.findAll('tr')[2:]:
      if arg in str(j):
        descript.append(j.text)

      else:
        #print('comeca', j.findAll('td')[4].text, 'termina')
        attack = {
              'lvl': j.findAll('td')[0].text,
              'attack_name': j.findAll('td')[1].text,
              'attack_type': str(j.find_all('img')[0]).split("/",)[3].split(".")[0],
              'category': str(j.find_all('img')[1]).split("/",)[3].split(".")[0],
              'att': j.find_all('td')[4].text,
              'acc': j.find_all('td')[5].text,
              'pp': j.find_all('td')[6].text,
              'effect': j.find_all('td')[7].text,
              }
        moves_xy.update({index:attack})
        index+=1
        print(attack)

for chave, valor in zip(moves_xy.keys(), descript):
        moves_xy[chave]['description'] = valor

SyntaxError: ignored

In [ ]:
moves

In [ ]:
def get_moves(type):
  table = soup.findAll(class_='dextable')
  arg = 'colspan="8"'
  moves = {}
  descript = []
  index = 0
  for i in table:
    #print(i.find('tr'))

    if type in str(i.find('tr')):
      #print(i)
      for j in i.findAll('tr')[2:]:
        if arg in str(j):
          descript.append(j.text)

        else:
          #print('comeca', j.findAll('td')[4].text, 'termina')
          attack = {
                'lvl': j.findAll('td')[0].text,
                'attack_name': j.findAll('td')[1].text,
                'attack_type': str(j.find_all('img')[0]).split("/",)[3].split(".")[0],
                'category': str(j.find_all('img')[1]).split("/",)[3].split(".")[0],
                'att': j.find_all('td')[4].text,
                'acc': j.find_all('td')[5].text,
                'pp': j.find_all('td')[6].text,
                'effect': j.find_all('td')[7].text,
                }
          moves.update({index:attack})
          index+=1
          #print(attack)

        for chave, valor in zip(moves.keys(), descript):
                moves[chave]['description'] = valor
        return moves
      else:
        return None

In [ ]:
def get_moves_dex(soup):


  moves_tm = get_moves('TM &amp; HM Attacks')

    moves_xy = get_moves('X &amp; Y Level Up')
    moves_oras = get_moves('ΩRαS Level Up')
  except:
    moves_xy = get_moves('Generation VI Level Up')

  moves = {
    'xy': moves_xy,
    'oras': moves_oras,
    'tm/hm': moves_tm
  }

  return moves

In [ ]:
eita = get_moves_dex(soup)
eita

In [ ]:
table = soup.findAll(class_='dextable')
arg = 'colspan="8"'
moves = []
descript = []
index = 0
for i in table:
  if 'Stats' in str(i.find('tr')):
    #print(i)
    for j in i.findAll('tr')[2:]:
      print(j.text)
        #print(attack)

#for chave, valor in zip(moves.keys(), descript):
 #       moves[chave]['description'] = valor

In [ ]:
moves

In [ ]:
def find_egg_group(soup):
    tag = soup.find_all('td', class_='fooinfo')
    egg = tag[15].find_all('tr')
    groups = []
    for i in egg:
        groups.append(i.find('option').text)
    list_keys = []
    for i in range(len(groups)):
      list_keys.append(f"egg_group_{i+1}")
    egg_group = dict(zip(list_keys, groups))
    return egg_group

In [ ]:
tag = soup.find_all('td', class_='fooinfo')
egg = tag[15].find_all('tr')
for i in egg:
  print(i.find('option').text)

In [ ]:
tag[15]

In [ ]:
table = soup.find(class_="dextable")
for i in table.next_siblings:
  #if 'Generation' in i.find_next('td'):
  #print(i.find('td'))
  a = i.find('tr')
  arg = 'colspan="8"'
  if 'Egg Groups' in str(a):
    print(i.find('tr'))

    for k in i.find_all('tr'):
      print(k.find('option'))